In [51]:
import matplotlib
matplotlib.use("Qt5Agg")

In [52]:
import numpy as np
import pandas as pd
import sys
import tensorflow as tf
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import BatchNormalization, Bidirectional
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from scipy.signal import find_peaks, savgol_filter
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from helpers import create_df
from helpers import get_peaks_and_valleys
from helpers import plot_df
from helpers import get_num_reps
from helpers import smooth_and_resample

from tcn import TCN
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, GRU
from mpl_toolkits.mplot3d import Axes3D

In [53]:
# 0 = bicep curl, 1 = shoulder press, 2 = row, 3 = rdl, 4 = squat 
training_files = {"../data2/bicep_curl/bicep_adam_set1_reps10_1.csv": 0, 
                  "../data2/bicep_curl/bicep_andrew_set2_reps10_1.csv": 0, 
                  "../data2/bicep_curl/bicep_anuj_set1_rep10_1.csv": 0, 
                  "../data2/bicep_curl/bicep_jake_set1_reps10_1.csv": 0, 
                  "../data2/bicep_curl/bicep_kamyar_set1_reps10_1.csv": 0, 
                  "../data2/bicep_curl/bicep_mike_set1_reps10_1.csv": 0, 
                  "../data2/bicep_curl/bicep_suzan_set1_reps10_1.csv": 0, 
                  "../data2/bicep_curl/bicep_udai_set1_reps10_1.csv": 0, 
                  "../data2/shoulder_press/shoulder_adam_set1_reps10_1.csv": 1, 
                  "../data2/shoulder_press/shoulder_andrew_set1_reps10_1.csv": 1, 
                  "../data2/shoulder_press/shoulder_anuj_set1_rep10_1.csv": 1, 
                  "../data2/shoulder_press/shoulder_kamyar_set1_reps10_1.csv": 1, 
                  "../data2/shoulder_press/shoulder_mike_set1_reps10_1.csv": 1, 
                  "../data2/shoulder_press/shoulder_suzan_set1_reps10_1.csv": 1, 
                  "../data2/shoulder_press/shoulder_udai_set1_reps10_1.csv": 1, 
                  "../data2/shoulder_press/shoulder_suzan_set3_reps10_1.csv": 1,
                  "../data2/row/row_adam_set1_reps10_1.csv": 2, 
                  "../data2/row/row_andrew_set1_reps10_1.csv": 2, 
                  "../data2/row/row_anuj_set1_rep10_1.csv": 2, 
                  "../data2/row/row_jake_set1_reps12_1.csv": 2, 
                  "../data2/row/row_kamyar_set1_reps10_1.csv": 2, 
                  "../data2/row/row_mike_set1_reps10_1.csv": 2, 
                  "../data2/row/row_suzan_set1_reps10_1.csv": 2, 
                  "../data2/row/row_udai_set1_reps12_1.csv": 2, 
                  "../data2/dead/dead_adam_set1_reps10_1.csv": 3, 
                  "../data2/dead/dead_andrew_set1_reps10_1.csv": 3, 
                  "../data2/dead/dead_anuj_set1_rep10_1.csv": 3, 
                  "../data2/dead/dead_kamyar_set1_reps10_1.csv": 3, 
                  "../data2/dead/dead_mike_set1_reps10_1.csv": 3, 
                  "../data2/squat/squat_adam_set1_reps10_1.csv": 4, 
                  "../data2/squat/squat_andrew_set1_reps10_1.csv": 4, 
                  "../data2/squat/squat_anuj_set1_rep10_1.csv": 4, 
                  "../data2/squat/squat_jake_set1_reps10_1.csv": 4, 
                  "../data2/squat/squat_kamyar_set1_reps10_1.csv": 4, 
                  "../data2/squat/squat_mike_set1_reps10_1.csv": 4, 
                  "../data2/squat/squat_suzan_set1_reps10_1.csv": 4, 
                  "../data2/squat/squat_udai_set1_reps10_1.csv": 4,
                  "../data2/bicep_curl/bicep_udai_set3_reps10_1.csv": 0, 
                  "../data2/shoulder_press/shoulder_udai_set3_reps10_1.csv": 1, 
                  "../data2/row/row_udai_set3_reps10_1.csv": 2, 
                  "../data2/dead/dead_udai_set1_reps10_1.csv": 3, 
                  "../data2/squat/squat_udai_set3_reps10_1.csv": 4,
                  "../data2/newDead/newdead_udai_set1_reps10_1.csv": 3} 
train_labels = [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 0, 1, 2, 3, 4, 3] 

validation_files = [
                    "../data2/bicep_curl/bicep_udai_set3_reps10_1.csv", 
                    "../data2/shoulder_press/shoulder_udai_set3_reps10_1.csv", 
                    "../data2/row/row_udai_set3_reps10_1.csv", 
                    "../data2/dead/dead_udai_set1_reps10_1.csv", 
                    "../data2/squat/squat_udai_set3_reps10_1.csv"] 

validation_labels = [0, 1, 2, 3, 4] 

test_files = {"../data2/bicep_curl/bicep_adam_set2_reps10_1.csv": 0, 
              "../data2/bicep_curl/bicep_andrew_set3_reps10_1.csv": 0, 
              "../data2/bicep_curl/bicep_anuj_set2_rep10_1.csv": 0, 
              "../data2/bicep_curl/bicep_jake_set2_reps10_1.csv": 0, 
              "../data2/bicep_curl/bicep_kamyar_set2_reps10_1.csv": 0, 
              "../data2/bicep_curl/bicep_mike_set2_reps10_1.csv": 0, 
              "../data2/bicep_curl/bicep_suzan_set2_reps10_1.csv": 0, 
              "../data2/bicep_curl/bicep_udai_set2_reps10_1.csv": 0, 
              "../data2/shoulder_press/shoulder_adam_set2_reps10_1.csv": 1, 
              "../data2/shoulder_press/shoulder_andrew_set2_reps10_1.csv": 1, 
              "../data2/shoulder_press/shoulder_anuj_set2_rep10_1.csv": 1, 
              "../data2/shoulder_press/shoulder_kamyar_set2_reps10_1.csv": 1, 
              "../data2/shoulder_press/shoulder_mike_set2_reps10_1.csv": 1, 
              "../data2/shoulder_press/shoulder_suzan_set2_reps8_1.csv": 1, 
              "../data2/shoulder_press/shoulder_suzan_set4_reps10_1.csv": 1,
              "../data2/shoulder_press/shoulder_udai_set2_reps9_1.csv": 1, 
              "../data2/row/row_adam_set2_reps10_1.csv": 2, 
              "../data2/row/row_andrew_set2_reps10_1.csv": 2, 
              "../data2/row/row_anuj_set2_rep10_1.csv": 2, 
              "../data2/row/row_jake_set2_reps12_1.csv": 2, 
              "../data2/row/row_kamyar_set2_reps10_1.csv": 2, 
              "../data2/row/row_mike_set2_reps10_1.csv": 2, 
              "../data2/row/row_suzan_set2_reps12_1.csv": 2, 
              "../data2/row/row_udai_set2_reps12_1.csv": 2, 
              "../data2/dead/dead_adam_set2_reps10_1.csv": 3, 
              "../data2/dead/dead_andrew_set2_reps10_1.csv": 3,
              "../data2/dead/dead_anuj_set2_rep10_1.csv": 3, 
              "../data2/dead/dead_kamyar_set2_reps10_1.csv": 3, 
              "../data2/dead/dead_mike_set2_reps10_1.csv": 3, 
              "../data2/squat/squat_adam_set2_reps10_1.csv": 4, 
              "../data2/squat/squat_andrew_set2_reps10_1.csv": 4, 
              "../data2/squat/squat_anuj_set2_rep10_1.csv": 4, 
              "../data2/squat/squat_jake_set2_reps10_1.csv": 4, 
              "../data2/squat/squat_kamyar_set2_reps10_1.csv": 4, 
              "../data2/squat/squat_mike_set2_reps10_1.csv": 4, 
              "../data2/squat/squat_suzan_set2_reps10_1.csv": 4, 
              "../data2/squat/squat_udai_set2_reps10_1.csv": 4,
            #   "../data2/newDead/newdead_udai_set2_reps10_1.csv": 3
              } 

test_labels = [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3]

In [54]:
%matplotlib qt


fig = plt.figure()
ax = fig.add_subplot(projection='3d')

n = 100

bicep_xs = np.empty(shape=[0, 1])
bicep_ys = np.empty(shape=[0, 1])
bicep_zs = np.empty(shape=[0, 1])

shoulder_xs = np.empty(shape=[0, 1])
shoulder_ys = np.empty(shape=[0, 1])
shoulder_zs = np.empty(shape=[0, 1])

row_xs = np.empty(shape=[0, 1])
row_ys = np.empty(shape=[0, 1])
row_zs = np.empty(shape=[0, 1])

rdl_xs = np.empty(shape=[0, 1])
rdl_ys = np.empty(shape=[0, 1])
rdl_zs = np.empty(shape=[0, 1])

squat_xs = np.empty(shape=[0, 1])
squat_ys = np.empty(shape=[0, 1])
squat_zs = np.empty(shape=[0, 1])

for file, label in training_files.items():
    df = create_df(file)
    df = smooth_and_resample(df)

    xs = df.loc[:, "accel_x"].values
    ys = df.loc[:, "accel_y"].values
    zs = df.loc[:, "accel_z"].values

    # print(f"bicep_xs: {bicep_xs}")
    # print(f"xs: {xs}")

    if label == 0:
        bicep_xs = np.concatenate((bicep_xs, np.reshape(xs, shape=[len(xs), 1])))
        bicep_ys = np.concatenate((bicep_ys, np.reshape(ys, shape=[len(ys), 1])))
        bicep_zs = np.concatenate((bicep_zs, np.reshape(zs, shape=[len(zs), 1])))
    elif label == 1:
        shoulder_xs = np.concatenate((shoulder_xs, np.reshape(xs, shape=[len(xs), 1])))
        shoulder_ys = np.concatenate((shoulder_ys, np.reshape(ys, shape=[len(ys), 1])))
        shoulder_zs = np.concatenate((shoulder_zs, np.reshape(zs, shape=[len(zs), 1])))
    elif label == 2:
        row_xs = np.concatenate((row_xs, np.reshape(xs, shape=[len(xs), 1])))
        row_ys = np.concatenate((row_ys, np.reshape(ys, shape=[len(ys), 1])))
        row_zs = np.concatenate((row_zs, np.reshape(zs, shape=[len(zs), 1])))
    elif label == 3:
        rdl_xs = np.concatenate((rdl_xs, np.reshape(xs, shape=[len(xs), 1])))
        rdl_ys = np.concatenate((rdl_ys, np.reshape(ys, shape=[len(ys), 1])))
        rdl_zs = np.concatenate((rdl_zs, np.reshape(zs, shape=[len(zs), 1])))
    elif label == 4:
        squat_xs = np.concatenate((squat_xs, np.reshape(xs, shape=[len(xs), 1])))
        squat_ys = np.concatenate((squat_ys, np.reshape(ys, shape=[len(ys), 1])))
        squat_zs = np.concatenate((squat_zs, np.reshape(zs, shape=[len(zs), 1])))


ax.scatter(bicep_xs, bicep_ys, bicep_zs, c='red')
ax.scatter(shoulder_xs, shoulder_ys, shoulder_zs, c='blue')
ax.scatter(row_xs, row_ys, row_zs, c='green')
ax.scatter(rdl_xs, rdl_ys, rdl_zs, c='orange')
ax.scatter(squat_xs, squat_ys, squat_zs, c='purple')



plt.show()

../data2/bicep_curl/bicep_adam_set1_reps10_1.csv
   timestamp  accel_x  accel_y  accel_z  gyro_x  gyro_y  gyro_z
0     304079   -0.834    3.319    9.444   0.098  -0.054   0.012
1     304082   -0.824    3.326    9.452   0.098  -0.054   0.012
2     304085   -0.824    3.326    9.452   0.097  -0.053   0.012
3     304089   -0.824    3.326    9.452   0.097  -0.053   0.012
4     304110   -0.846    3.269    9.446   0.122  -0.056   0.007
../data2/bicep_curl/bicep_andrew_set2_reps10_1.csv
   timestamp  accel_x  accel_y  accel_z  gyro_x  gyro_y  gyro_z
0    1783092    0.522    4.887   -8.289  -0.454  -0.057  -0.042
1    1783096    0.536    4.859   -8.264  -0.519  -0.046  -0.043
2    1783099    0.536    4.859   -8.264  -0.519  -0.046  -0.043
3    1783103    0.536    4.859   -8.264  -0.519  -0.046  -0.043
4    1783126    0.474    4.851   -8.181  -0.552  -0.057  -0.049
../data2/bicep_curl/bicep_anuj_set1_rep10_1.csv
   timestamp  accel_x  accel_y  accel_z  gyro_x  gyro_y  gyro_z
0     343675   -0.31

2025-11-23 20:10:14.623 python[40104:1557193] +[IMKClient subclass]: chose IMKClient_Modern
2025-11-23 20:10:14.623 python[40104:1557193] +[IMKInputSession subclass]: chose IMKInputSession_Modern


: 